In [224]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.init as I
from torch.nn import Sigmoid 
class RBM(nn.Module):

    def __init__(self,
                 visible_size,
                 hidden_size):
        super(RBM, self).__init__()
        self.input_layer = visible_size
        self.output_layer = hidden_size
        
        self.feed = nn.Linear(visible_size, hidden_size,bias=False)  

    def forward(self, x):
        h = Sigmoid()(self.feed(x))        # x -> h 

        x_reconstructed = torch.matmul(h,self.feed.weight)
        
        h_reconstructed = Sigmoid()(self.feed(x_reconstructed))# h-> x'

        return h,x_reconstructed,h_reconstructed





In [225]:
data = torch.rand(5210,257)
model = RBM(257, 200)
model.feed.weight.requires_grad = True

### Contrastive divergence

In [241]:
lr = 0.001
for param in model.parameters():
  weight_update = (torch.matmul(data.t(),h) - torch.matmul(x_reconstructed.t(),h_reconstructed)).t()
  param.data.sub_(lr*weight_update)

In [242]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=lr)


## Training

In [243]:
n_epochs = 100

In [244]:
trainloader = torch.utils.data.DataLoader(data,batch_size=64)

In [245]:
a = model.feed.weight
a

Parameter containing:
tensor([[-0.0334,  0.0095,  0.0344,  ...,  0.0145, -0.0590, -0.0551],
        [ 0.0574, -0.0466, -0.0293,  ..., -0.0191,  0.0455, -0.0550],
        [ 0.0291,  0.0243, -0.0059,  ..., -0.0087, -0.0183,  0.0062],
        ...,
        [-0.0382,  0.0549,  0.0325,  ...,  0.0503,  0.0478, -0.0399],
        [ 0.0010, -0.0376, -0.0361,  ..., -0.0348,  0.0328,  0.0363],
        [ 0.0516,  0.0589, -0.0569,  ...,  0.0311, -0.0141,  0.0158]],
       requires_grad=True)

In [ ]:
losses = []
#model = RBM(257, 200)

for epoch in range(150):  # loop over the dataset multiple times

    running_loss = 0.0
    for i,data in enumerate(trainloader):

        # zero the parameter gradients
        optimizer.zero_grad()
        # get the inputs; data is a list of [inputs, labels]
        h,x_reconstructed,h_reconstructed = model(data)

        # forward + backward + optimize
        #loss = criterion(torch.matmul(data.t(),h),torch.matmul(x_reconstructed.t(),h_reconstructed))
        loss = criterion(x_reconstructed,data)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    losses.append(loss)
    print(loss)

print('Finished Training')

In [ ]:
b = model.feed.weight
b